# Sampling from a diffusion model

<!--- @wandbcode{dlai_03} -->

이 노트북에서는 이전에 학습된 확산 모델에서 샘플을 추출합니다.
- DDPM과 DDIM 샘플러의 샘플을 비교해 보겠습니다.
- 조건부 확산 모델을 사용하여 혼합 샘플 시각화하기

In [1]:
from pathlib import Path
from types import SimpleNamespace
import torch
import torch.nn.functional as F
import numpy as np
from utilities import *

import wandb

In [2]:
wandb.login(anonymous="allow")

wandb: Currently logged in as: kimwooglae. Use `wandb login --relogin` to force relogin


True

# Setting Things Up

In [3]:
# Wandb Params
MODEL_ARTIFACT = "dlai-course/model-registry/SpriteGen:latest"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

config = SimpleNamespace(
    # hyperparameters
    num_samples = 30,
    
    # ddpm sampler hyperparameters
    timesteps = 500,
    beta1 = 1e-4,
    beta2 = 0.02,
    
    # ddim sampler hp
    ddim_n = 25,
    
    # network hyperparameters
    height = 16,
)

이전 노트북에서 최상의 모델을 wandb 아티팩트(실행 중에 파일을 저장하는 방식)로 저장했습니다. 이제 wandb에서 모델을 로드하고 샘플링 루프를 설정하겠습니다.

In [4]:
def load_model(model_artifact_name):
    "Load the model from wandb artifacts"
    api = wandb.Api()
    artifact = api.artifact(model_artifact_name, type="model")
    model_path = Path(artifact.download())

    # recover model info from the registry
    producer_run = artifact.logged_by()

    # load the weights dictionary
    model_weights = torch.load(model_path/"context_model.pth", 
                               map_location="cpu")

    # create the model
    model = ContextUnet(in_channels=3, 
                        n_feat=producer_run.config["n_feat"], 
                        n_cfeat=producer_run.config["n_cfeat"], 
                        height=producer_run.config["height"])
    
    # load the weights into the model
    model.load_state_dict(model_weights)

    # set the model to eval mode
    model.eval()
    return model.to(DEVICE)

In [5]:
nn_model = load_model(MODEL_ARTIFACT)

wandb:   1 of 1 files downloaded.  


## Sampling

생성된 샘플을 wandb에 기록합니다.

In [15]:
_, sample_ddpm_context = setup_ddpm(config.beta1, 
                                    config.beta2, 
                                    config.timesteps, 
                                    DEVICE)

In [16]:
DEVICE

'cpu'

조건에 적용할 노이즈 세트와 컨텍스트 벡터를 정의해 보겠습니다.

In [17]:
# Noise vector
# x_T ~ N(0, 1), sample initial noise
noises = torch.randn(config.num_samples, 3, 
                     config.height, config.height).to(DEVICE)  

# A fixed context vector to sample from
ctx_vector = F.one_hot(torch.tensor([0,0,0,0,0,0,   # hero
                                     1,1,1,1,1,1,   # non-hero
                                     2,2,2,2,2,2,   # food
                                     3,3,3,3,3,3,   # spell
                                     4,4,4,4,4,4]), # side-facing 
                       5).to(DEVICE).float()

확산 과정에서 더 빠른 DDIM 샘플러를 가져와 보겠습니다.

In [18]:
sample_ddim_context = setup_ddim(config.beta1, 
                                 config.beta2, 
                                 config.timesteps, 
                                 DEVICE)

### Sampling:
이전과 같이 DDPM 샘플을 계산해 보겠습니다.

In [19]:
ddpm_samples, _ = sample_ddpm_context(nn_model, noises, ctx_vector)

  0%|          | 0/500 [00:00<?, ?it/s]

DDIM의 경우 `n` 파라미터로 스텝 크기를 제어할 수 있습니다:

In [10]:
ddim_samples, _ = sample_ddim_context(nn_model, 
                                      noises, 
                                      ctx_vector, 
                                      n=config.ddim_n)

  0%|          | 0/25 [00:00<?, ?it/s]

### 테이블에서 생성 시각화
Let's create a `wandb.Table` to store our generations

In [11]:
table = wandb.Table(columns=["input_noise", "ddpm", "ddim", "class"])

테이블에 행을 하나씩 추가할 수 있으며, 이미지를 `wandb.Image`로 캐스팅하여 UI에서 올바르게 렌더링할 수 있도록 할 수도 있습니다.

In [12]:
for noise, ddpm_s, ddim_s, c in zip(noises, 
                                    ddpm_samples, 
                                    ddim_samples, 
                                    to_classes(ctx_vector)):
    
    # add data row by row to the Table
    table.add_data(wandb.Image(noise),
                   wandb.Image(ddpm_s), 
                   wandb.Image(ddim_s),
                   c)

테이블을 W&B에 로깅하고, 컨텍스트 관리자로 `wandb.init`을 사용할 수도 있으므로 관리자를 종료할 때 실행이 완료되도록 할 수 있습니다.

In [13]:
with wandb.init(project="dlai_sprite_diffusion", 
                job_type="samplers_battle", 
                config=config):
    
    wandb.log({"samplers_table":table})